In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SequentialFeatureSelector
#from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
# Import training data
train_df = pd.read_csv("C:/Users/peiha/Documents/GitHub/ApplyAIGroupProject/original_train.csv")
train_df.head(2)

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1


In [3]:
print("Number of rows in training dataset:", len(train_df))

Number of rows in training dataset: 75757


In [4]:
# Get NaN values in each column
train_df.isna().sum()

Year_Factor                      0
State_Factor                     0
building_class                   0
facility_type                    0
floor_area                       0
                             ...  
direction_peak_wind_speed    41811
max_wind_speed               41082
days_with_fog                45796
site_eui                         0
id                               0
Length: 64, dtype: int64

In [5]:
#pull numeric features
df_with_numeric_features = train_df.select_dtypes(exclude=['object'])

#print("Numeric features:")
#for col in df_with_numeric_features:
#    print(f"{col}")

In [6]:
# Pull non-numeric features
df_with_non_numeric_features = train_df.select_dtypes(include=['object'])

print("Non-numeric columns:")
for col in df_with_non_numeric_features:
    print(f"{col}")

Non-numeric columns:
State_Factor
building_class
facility_type


In [7]:
# Remove duplicates
column_names = train_df.columns.tolist()
column_names.remove('id')
train_df.drop_duplicates(subset=column_names, keep='last', inplace=True)
print("Number of rows after removing duplicates", len(train_df))

Number of rows after removing duplicates 75718


In [8]:
# Filter outliers

features_considered = ['Year_Factor', 'floor_area','year_built', 'energy_star_rating', 'ELEVATION','cooling_degree_days',\
                       'heating_degree_days','avg_temp','site_eui']
# calculate z-scores for numeric features
z_scores = zscore(train_df[features_considered], nan_policy='omit')

# convert to absolute values
abs_z_scores = np.abs(z_scores)
# filter based on |z-score| less than 3
filtered_entries = (abs_z_scores < 3).all(axis=1)

print("Outliers present in the dataset")
train_df.loc[filtered_entries == False]

Outliers present in the dataset


,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
5,1,State_1,Commercial,Warehouse_Selfstorage,119900.0,1956.0,NaN,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,6.597715,5
13,1,State_1,Commercial,Data_Center,616793.0,1955.0,13.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,608.839519,13
14,1,State_1,Commercial,Commercial_Other,53000.0,1924.0,NaN,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,2.699413,14
15,1,State_1,Commercial,Office_Uncategorized,90045.0,NaN,NaN,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,24.600000,15
17,1,State_1,Commercial,Office_Uncategorized,128800.0,1926.0,NaN,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,33.580371,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75749,6,State_11,Commercial,Commercial_Other,47237.0,1948.0,NaN,63.1,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,126.285151,75749
75750,6,State_11,Commercial,Commercial_Other,72000.0,2013.0,NaN,63.1,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,40.525884,75750
75754,6,State_11,Commercial,Commercial_Other,28072.0,1917.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,48.404398,75754
75755,6,State_11,Commercial,Commercial_Other,53575.0,2012.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,592.022750,75755


In [9]:
df_without_outliers = train_df[filtered_entries]
print('Number of rows after filtering outliers:', len(df_without_outliers))
train_df = df_without_outliers;
df_without_outliers.head()

Number of rows after filtering outliers: 44589


,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [10]:
# Handle missing values

#Find columns which have missing values
train_df.isna().sum()[train_df.isna().sum()>0]

direction_max_wind_speed     23955
direction_peak_wind_speed    24512
max_wind_speed               23955
days_with_fog                27915
dtype: int64

In [11]:
#Delete columns that have missing values
train_df = train_df.drop(columns=['id','direction_max_wind_speed','direction_peak_wind_speed',\
                                  'max_wind_speed','days_with_fog'])

In [12]:
#Delete rows that have missing values
#train_df = train_df.dropna(subset = ['id','direction_max_wind_speed','direction_peak_wind_speed',\
#                                     'max_wind_speed','days_with_fog']).reset_index(drop = True)

In [13]:
# get count of missing value in train_df
train_df.columns[train_df.isna().any()].tolist()

[]

In [14]:
print("Number of rows in training data after filtering missing values:", len(train_df))
train_df.head()

Number of rows in training data after filtering missing values: 44589


,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,site_eui
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,56.972603,0,0,0,0,14,0,0,0,248.682615
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,56.972603,0,0,0,0,14,0,0,0,26.500150
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,56.972603,0,0,0,0,14,0,0,0,24.693619
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,56.972603,0,0,0,0,14,0,0,0,48.406926
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,56.972603,0,0,0,0,14,0,0,0,3.899395


In [15]:
# Convert categorical variables to numerical

# grab non-numeric features
df_with_non_numeric_features = train_df.select_dtypes(include=['object'])
print("Non-numeric columns:")
for col in df_with_non_numeric_features:
    print(f"{col}")

Non-numeric columns:
State_Factor
building_class
facility_type


In [16]:
train_df['State_Factor'].value_counts()

State_6     29231
State_11     4678
State_2      3749
State_4      2912
State_8      2096
State_1      1914
State_10        9
Name: State_Factor, dtype: int64

In [17]:
train_df['building_class'].value_counts()

Residential    25392
Commercial     19197
Name: building_class, dtype: int64

In [18]:
#train_df['facility_type'].value_counts()

In [19]:
# OneHotEncoding 'State_Factor'
encoder = OneHotEncoder(handle_unknown = 'ignore')
encoder.fit(np.array(train_df['State_Factor'].tolist()).reshape(-1, 1))

OneHotEncoder(handle_unknown='ignore')

In [20]:
# Create a Pandas DataFrame of the hot encoded column
transformed = encoder.transform(np.array(train_df['State_Factor'].tolist()).reshape(-1, 1)).toarray()
ohe_df_train = pd.DataFrame(transformed, columns=encoder.get_feature_names())

In [21]:
# Concat with original data
train_df.reset_index(drop=True, inplace=True)
ohe_df_train.reset_index(drop=True, inplace=True)

train_df = pd.concat([train_df, ohe_df_train], axis=1)
train_df = train_df.drop(['State_Factor'], axis=1)
train_df.head()

,Year_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_100F,days_above_110F,site_eui,x0_State_1,x0_State_10,x0_State_11,x0_State_2,x0_State_4,x0_State_6,x0_State_8
0,1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,68,...,0,0,248.682615,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,68,...,0,0,26.500150,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,68,...,0,0,24.693619,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,68,...,0,0,48.406926,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,68,...,0,0,3.899395,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Encode ordinal values

encode_categorical_values = {"Residential":0, "Commercial":1}
train_df['building_class'] = train_df['building_class'].replace(encode_categorical_values) 
train_df.head()

,Year_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_100F,days_above_110F,site_eui,x0_State_1,x0_State_10,x0_State_11,x0_State_2,x0_State_4,x0_State_6,x0_State_8
0,1,1,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,68,...,0,0,248.682615,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,68,...,0,0,26.500150,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,68,...,0,0,24.693619,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,68,...,0,0,48.406926,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,68,...,0,0,3.899395,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Creating cleaned_train.csv and cleaned_test.csv from the original_train.csv, the spliting ratio is 1:9
training_data, testing_data = train_test_split(train_df, test_size=0.1, random_state=25)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 40130
No. of testing examples: 4459


In [24]:
# Save the cleaned_train.csv and the cleaned_test.csv
training_data.to_csv('cleaned_train.csv', index=False)
testing_data.to_csv('cleaned_test.csv', index=False)

In [25]:
training_data.head()

,Year_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_100F,days_above_110F,site_eui,x0_State_1,x0_State_10,x0_State_11,x0_State_2,x0_State_4,x0_State_6,x0_State_8
36498,6,0,Multifamily_Uncategorized,76772.0,1931.0,59.0,25.0,11,34.451613,59,...,0,0,87.859290,0.0,0.0,0.0,0.0,0.0,1.0,0.0
32462,6,0,Multifamily_Uncategorized,84757.0,2007.0,46.0,42.7,11,34.451613,59,...,0,0,63.924863,0.0,0.0,0.0,0.0,0.0,1.0,0.0
36500,6,0,Multifamily_Uncategorized,65862.0,1930.0,45.0,25.0,11,34.451613,59,...,0,0,104.314208,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2325,4,1,Office_Uncategorized,240578.0,2000.0,69.0,47.5,3,28.290323,59,...,0,0,72.833560,0.0,0.0,0.0,1.0,0.0,0.0,0.0
37607,6,0,Multifamily_Uncategorized,85000.0,1950.0,74.0,24.4,11,34.451613,59,...,0,0,71.304645,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
testing_data.head()

,Year_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_100F,days_above_110F,site_eui,x0_State_1,x0_State_10,x0_State_11,x0_State_2,x0_State_4,x0_State_6,x0_State_8
11910,3,1,Office_Uncategorized,184405.0,1917.0,72.0,15.2,11,35.080645,61,...,0,0,56.637582,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15830,4,1,Office_Uncategorized,736974.0,1960.0,55.0,42.7,4,28.596774,58,...,0,0,106.618289,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10780,2,1,Lodging_Hotel,233230.0,1989.0,13.0,3.4,13,37.322581,62,...,0,0,96.635246,0.0,0.0,0.0,0.0,0.0,1.0,0.0
40147,5,0,2to4_Unit_Building,25301.0,1988.0,80.0,18.3,28,45.419355,63,...,0,0,26.990755,0.0,0.0,1.0,0.0,0.0,0.0,0.0
43181,6,0,5plus_Unit_Building,28350.0,1999.0,96.0,26.5,28,43.451613,56,...,0,0,44.738108,0.0,0.0,1.0,0.0,0.0,0.0,0.0
